In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.handspeak.com/word/most-used/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

words = []
for li in soup.select('ol li'):
    words.append(li.text.strip())

print(words)

In [ ]:
import os

root_dir = "datasetMVP"

if not os.path.exists(root_dir):
    os.makedirs(root_dir)

for word in words:
    sub_dir = os.path.join(root_dir, word)
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)

In [ ]:
import os
import numpy as np
import math

dataset_path = "MP_data"

def rotate_keypoints(keypoints, angle):
    # Get the center of mass of the keypoints
    center = keypoints.mean(axis = 0)[:2]
    angle = math.radians(angle)
    c, s = np.cos(angle), np.sin(angle)
    R = np.array(((c,-s), (s, c)))
    # Translate points to center
    keypoints[:,:2] = keypoints[:,:2] - center
    # Rotate the keypoints
    keypoints[:,:2] = np.dot(keypoints[:,:2], R)
    # Translate points back
    keypoints[:,:2] = keypoints[:,:2] + center
    return keypoints

for sign in os.listdir(dataset_path):
    sign_path = os.path.join(dataset_path, sign)
    for video in range(30):
        video_path = os.path.join(sign_path, str(video))
        for frame in range(30):
            frame_path = os.path.join(video_path, str(frame) + ".npy")
            keypoints = np.load(frame_path)
            angle = np.random.uniform(-10, 10)
            keypoints = rotate_keypoints(keypoints, angle)
            new_file_name = str(frame) + "_data_aug_rotate" + ".npy"
            new_file_path = os.path.join(video_path, new_file_name)
            np.save(new_file_path, keypoints)

In [ ]:
import matplotlib.pyplot as plt

# Load the numpy array
keypoints = np.load("keypoints.npy")

# Extract x and y coordinates of the keypoints
x = keypoints[:, 0]
y = keypoints[:, 1]

fig = plt.figure()

plt.scatter(x, y)

plt.plot(x[:21], y[:21], 'r-')   # left hand
plt.plot(x[21:42], y[21:42], 'b-') # right hand
plt.plot(x[42:], y[42:], 'g-')    # pose

plt.show()

In [ ]:
import os
import torch
import numpy as np

root_path = "MVP_dataset"

labels = ['hello', 'help', 'sorry']

for label in labels:
    label_path = os.path.join(root_path, label)
    
    subfolders = os.listdir(label_path)
    
    for subfolder in subfolders:
        subfolder_path = os.path.join(label_path, subfolder)
        
        np_arrays = []
        
        for i in range(48):
            np_file_path = os.path.join(subfolder_path, f"{i}.npy")
            
            np_array = np.load(np_file_path)
            
            np_arrays.append(torch.from_numpy(np_array))
        
        tensor = torch.cat(np_arrays, dim=0)
        
        tensor_path = os.path.join(label_path, f"{subfolder}.pt")
        
        torch.save(tensor, tensor_path)

In [ ]:
import time

labels = ['hello', 'sorry', 'help']

for label in labels:
    label_folder = os.path.join('FinalFinalDataset', label)
    os.makedirs(label_folder, exist_ok=True)
    for i in range(15):
        video_folder = os.path.join(label_folder, str(i))
        os.makedirs(video_folder, exist_ok=True)
        holistic = get_holistic_model()
        cap = cv2.VideoCapture(0)
        for j in range(48):
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            keypoints = extract_keypoints(results)
            cv2.putText(frame, f"Label: {label}, Video: {i}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.imshow('OpenCV Feed', image)
            keypoints_path = os.path.join(video_folder, f"{j}.pt")
            torch.save(keypoints, keypoints_path)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            time.sleep(0.1)
        cap.release()
        cv2.destroyAllWindows()
        time.sleep(2)